# Auto Generated Agent Chat: Using RetrieveChat for Retrieve Augmented Code Generation and Question Answering

RetrieveChat is a convesational framework for retrieve augmented code generation and question answering. In this notebook, we demonstrate how to utilize RetrieveChat to generate code and answer questions based on customized documentations that are not present in the LLM's training dataset. RetrieveChat uses the `RetrieveAssistantAgent` and `RetrieveUserProxyAgent`, which is similar to the usage of `AssistantAgent` and `UserProxyAgent` in other notebooks (e.g., [Automated Task Solving with Code Generation, Execution & Debugging](https://github.com/microsoft/FLAML/blob/main/notebook/autogen_agentchat_auto_feedback_from_code_execution.ipynb)). Essentially,`RetrieveAssistantAgent` and  `RetrieveUserProxyAgent` implements a different auto reply mechanism corresponding to the RetrieveChat prompts.

## Requirements

FLAML requires `Python>=3.8`. To run this notebook example, please install flaml with the [mathchat] option.
```bash
pip install flaml[retrievechat]
```

In [1]:
# %pip install flaml[retrievechat]~=2.0.0rc4

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/FLAML/docs/reference/autogen/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.


In [3]:
from flaml import autogen

config_list = autogen.config_list_from_json(
    env_or_file=".config.local",
    file_location=".",
    filter_dict={
        "model": {
            "gpt-4",
            "gpt4",
            "gpt-4-32k",
            "gpt-4-32k-0314",
            "gpt-35-turbo",
            "gpt-3.5-turbo",
        }
    },
)

assert len(config_list) > 0
print("models to use: ", [config_list[i]["model"] for i in range(len(config_list))])

models to use:  ['gpt-4']


It first looks for environment variable "OAI_CONFIG_LIST" which needs to be a valid json string. If that variable is not found, it then looks for a json file named "OAI_CONFIG_LIST". It filters the configs by models (you can filter by other keys as well). Only the gpt-4 and gpt-3.5-turbo models are kept in the list based on the filter condition.

The config list looks like the following:
```python
config_list = [
    {
        'model': 'gpt-4',
        'api_key': '<your OpenAI API key here>',
    },
    {
        'model': 'gpt-4',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
    {
        'model': 'gpt-3.5-turbo',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
]
```

If you open this notebook in colab, you can upload your files by clicking the file icon on the left panel and then choose "upload file" icon.

You can set the value of config_list in other ways you prefer, e.g., loading from a YAML file.

## Construct agents for RetrieveChat

We start by initialzing the `RetrieveAssistantAgent` and `RetrieveUserProxyAgent`. The system message needs to be set to "You are a helpful assistant." for RetrieveAssistantAgent. The detailed instructions are given in the user message. Later we will use the `RetrieveUserProxyAgent.generate_init_prompt` to combine the instructions and a math problem for an initial prompt to be sent to the LLM assistant.

In [4]:
from flaml.autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from flaml.autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
import chromadb

autogen.ChatCompletion.start_logging()

# 1. create an RetrieveAssistantAgent instance named "assistant"
assistant = RetrieveAssistantAgent(
    name="assistant", 
    system_message="You are a helpful assistant.",
    llm_config={
        "request_timeout": 600,
        "seed": 42,
        "config_list": config_list,
    },
)

# 2. create the RetrieveUserProxyAgent instance named "ragproxyagent"
# By default, the human_input_mode is "ALWAYS", which means the agent will ask for human input at every step. We set it to "NEVER" here.
# `docs_path` is the path to the docs directory. By default, it is set to "./docs". Here we generated the documentations from FLAML's docstrings.
# Navigate to the website folder and run `pydoc-markdown` and it will generate folder `reference` under `website/docs`.
# `chunk_token_size` is the chunk token size for the retrieve chat. By default, it is set to `max_tokens * 0.6`, here we set it to 2000.
ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    retrieve_config={
        "docs_path": "../website/docs/reference",
        "chunk_token_size": 2000,
        "model": config_list[0]["model"],
        "client": chromadb.PersistentClient(path="/tmp/chromadb"),
    },
)

### Example 1

Use RetrieveChat to help generate sample code and automatically run the code and fix errors if there is any.

Problem: Which API should I use if I want to use FLAML for a classification task and I want to train the model in 30 seconds. Use spark to parallel the training. Force cancel jobs if time limit is reached.

In [5]:
# reset the assistant. Always reset the assistant before starting a new conversation.
assistant.reset()

# given a problem, we use the ragproxyagent to generate a prompt to be sent to the assistant as the initial message.
# the assistant receives the message and generates a response. The response will be sent back to the ragproxyagent for processing.
# The conversation continues until the termination condition is met, in RetrieveChat, the termination condition when no human-in-loop is no code block detected.
# With human-in-loop, the conversation will continue until the user says "exit".
code_problem = "How can I use FLAML to perform a classification task and use spark to do parallel training. Train 30 seconds and force cancel jobs if time limit is reached."
ragproxyagent.initiate_chat(assistant, problem=code_problem, search_string="spark")  # search_string is used as an extra filter for the embeddings search, in this case, we only want to search documents that contain "spark".

ERROR:flaml.autogen.retrieve_utils:Collection flaml-docs already exists.


doc_ids:  [['doc_35', 'doc_40', 'doc_14', 'doc_21', 'doc_15', 'doc_51', 'doc_44', 'doc_52', 'doc_41', 'doc_45', 'doc_13', 'doc_38', 'doc_36', 'doc_8']]
Adding doc_id doc_35 to context.
Adding doc_id doc_40 to context.
Adding doc_id doc_14 to context.
ragproxyagent (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user. You should follow the following steps to answer a question:
Step 1, you estimate the user's intent based on the question and context. The intent can be a code generation task or
a QA task.
Step 2, you generate code or answer the question based on the intent.
You should leverage the context provided by the user as much as possible. If you think the context is not enough, you
can reply exactly "UPDATE CONTEXT" to ask the user to provide more contexts.
For code generation, you must obey the following rules:
You MUST NOT install any packages because all the packages needed are already

your 131072x1 screen size is bogus. expect trouble


23/08/03 22:00:19 WARN Utils: Your hostname, DESKTOP-TTKT4BA resolves to a loopback address: 127.0.1.1; using 172.31.215.15 instead (on interface eth0)
23/08/03 22:00:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


...

23/08/03 22:00:25 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
[flaml.tune.tune: 08-03 22:00:25] {729} INFO - Number of trials: 1/1000000, 1 RUNNING, 0 TERMINATED


[flaml.tune.tune: 08-03 22:00:34] {749} INFO - Brief result: {'pred_time': 6.8442026774088526e-06, 'wall_clock_time': 17.609724760055542, 'metric_for_logging': {'pred_time': 6.8442026774088526e-06}, 'val_loss': 0.6499914632462661, 'trained_estimator': <flaml.automl.model.LGBMEstimator object at 0x7f260cb084f0>}
[flaml.tune.tune: 08-03 22:00:34] {729} INFO - Number of trials: 2/1000000, 1 RUNNING, 1 TERMINATED
[flaml.tune.tune: 08-03 22:00:34] {749} INFO - Brief result: {'pred_time': 1.7034212748209633e-05, 'wall_clock_time': 17.8204128742218, 'metric_for_logging': {'pred_time': 1.7034212748209633e-05}, 'val_loss': 0.13431251746799158, 'trained_estimator': <flaml.automl.model.RandomForestEstimator object at 0x7f260cb08d30>}
[flaml.tune.tune: 08-03 22:00:34] {729} INFO - Number of trials: 3/1000000, 1 RUNNING, 2 TERMINATED


[flaml.tune.tune: 08-03 22:00:46] {749} INFO - Brief result: {'pred_time': 2.990245819091797e-05, 'wall_clock_time': 29.605072498321533, 'metric_for_logging': {'pred_time': 2.990245819091797e-05}, 'val_loss': 0.17314595016097636, 'trained_estimator': <flaml.automl.model.CatBoostEstimator object at 0x7f260cb08dc0>}
[flaml.tune.tune: 08-03 22:00:46] {729} INFO - Number of trials: 4/1000000, 1 RUNNING, 3 TERMINATED


/home/lijiang1/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
Time exceeded, canceled jobs                                        (0 + 1) / 1]



[flaml.automl.logger: 08-03 22:00:47] {2493} INFO - selected model: None
[flaml.automl.logger: 08-03 22:00:47] {2627} INFO - retrain rf for 0.0s
[flaml.automl.logger: 08-03 22:00:47] {2630} INFO - retrained model: RandomForestClassifier(criterion='entropy', max_features=0.5000000000000001,
                       max_leaf_nodes=4, n_estimators=4, n_jobs=-1,
                       random_state=12032022)
[flaml.automl.logger: 08-03 22:00:47] {1930} INFO - fit succeeded
[flaml.automl.logger: 08-03 22:00:47] {1931} INFO - Time taken to find the best model: 17.8204128742218
ragproxyagent (to assistant):

exitcode: 0 (execution succeeded)
Code output: 
None

--------------------------------------------------------------------------------
assistant (to ragproxyagent):

TERMINATE

--------------------------------------------------------------------------------


### Example 2

Use RetrieveChat to answer a question that is not related to code generation.

Problem: Who is the author of FLAML?

In [6]:
# reset the assistant. Always reset the assistant before starting a new conversation.
assistant.reset()

qa_problem = "Who is the author of FLAML?"
ragproxyagent.initiate_chat(assistant, problem=qa_problem)

doc_ids:  [['doc_58', 'doc_51', 'doc_35', 'doc_3', 'doc_22', 'doc_40', 'doc_14', 'doc_13', 'doc_59', 'doc_52', 'doc_1', 'doc_6', 'doc_28', 'doc_56', 'doc_29', 'doc_2', 'doc_55', 'doc_44', 'doc_19', 'doc_32']]
Adding doc_id doc_58 to context.
Adding doc_id doc_51 to context.
Adding doc_id doc_35 to context.
Adding doc_id doc_3 to context.
Adding doc_id doc_22 to context.
ragproxyagent (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user. You should follow the following steps to answer a question:
Step 1, you estimate the user's intent based on the question and context. The intent can be a code generation task or
a QA task.
Step 2, you generate code or answer the question based on the intent.
You should leverage the context provided by the user as much as possible. If you think the context is not enough, you
can reply exactly "UPDATE CONTEXT" to ask the user to provide more contexts.
For code ge

### Example 3

Use RetrieveChat to help generate sample code and ask for human-in-loop feedbacks.

Problem: how to build a time series forecasting model for stock price using FLAML?

In [7]:
# reset the assistant. Always reset the assistant before starting a new conversation.
assistant.reset()

# set `human_input_mode` to be `ALWAYS`, so the agent will ask for human input at every step.
ragproxyagent.human_input_mode = "ALWAYS"
code_problem = "how to build a time series forecasting model for stock price using FLAML?"
ragproxyagent.initiate_chat(assistant, problem=code_problem)

doc_ids:  [['doc_37', 'doc_35', 'doc_46', 'doc_40', 'doc_49', 'doc_51', 'doc_58', 'doc_48', 'doc_14', 'doc_39', 'doc_47', 'doc_41', 'doc_13', 'doc_60', 'doc_52', 'doc_59', 'doc_43', 'doc_10', 'doc_34', 'doc_33']]
Adding doc_id doc_37 to context.
Adding doc_id doc_35 to context.
Adding doc_id doc_46 to context.
Adding doc_id doc_40 to context.
Adding doc_id doc_49 to context.
ragproxyagent (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user. You should follow the following steps to answer a question:
Step 1, you estimate the user's intent based on the question and context. The intent can be a code generation task or
a QA task.
Step 2, you generate code or answer the question based on the intent.
You should leverage the context provided by the user as much as possible. If you think the context is not enough, you
can reply exactly "UPDATE CONTEXT" to ask the user to provide more contexts.
For co

/home/lijiang1/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/home/lijiang1/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/home/lijiang1/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/home/lijiang1/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...

>>>>>>>> EXECUTING CODE BLOCK 1 (inferred language is python)...


FileNotFoundError: [Errno 2] No such file or directory: 'your_stock_price_data.csv'

ragproxyagent (to assistant):

exitcode: 1 (execution failed)
Code output: 
None
None
[Errno 2] No such file or directory: 'your_stock_price_data.csv'

--------------------------------------------------------------------------------
exitcode: 1 (execution failed)
Code output: 
None
None
[Errno 2] No such file or directory: 'your_stock_price_data.csv'

--------------------------------------------------------------------------------
assistant (to ragproxyagent):

It seems I cannot execute the code since I don't have access to your stock price data file. However, you can follow the same steps I provided above to build a time series forecasting model for stock price using FLAML on your local machine or your preferred environment.

Please ensure you have the stock price data saved as a CSV file ("your_stock_price_data.csv") on your local machine. Change this file name in the code if needed and also adjust the "cutoff_date" to the desired date.

If you have any questions or issues, feel free

### Example 4

Use RetrieveChat to answer a question and ask for human-in-loop feedbacks.

Problem: Is there a function named `tune_automl` in FLAML?

In [8]:
# reset the assistant. Always reset the assistant before starting a new conversation.
assistant.reset()

# set `human_input_mode` to be `ALWAYS`, so the agent will ask for human input at every step.
ragproxyagent.human_input_mode = "ALWAYS"
qa_problem = "Is there a function named `tune_automl` in FLAML?"
ragproxyagent.initiate_chat(assistant, problem=qa_problem)

doc_ids:  [['doc_35', 'doc_14', 'doc_40', 'doc_13', 'doc_51', 'doc_58', 'doc_20', 'doc_26', 'doc_34', 'doc_22', 'doc_11', 'doc_59', 'doc_3', 'doc_56', 'doc_47', 'doc_53', 'doc_19', 'doc_28', 'doc_32', 'doc_46']]
Adding doc_id doc_35 to context.
Adding doc_id doc_14 to context.
Adding doc_id doc_40 to context.
ragproxyagent (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user. You should follow the following steps to answer a question:
Step 1, you estimate the user's intent based on the question and context. The intent can be a code generation task or
a QA task.
Step 2, you generate code or answer the question based on the intent.
You should leverage the context provided by the user as much as possible. If you think the context is not enough, you
can reply exactly "UPDATE CONTEXT" to ask the user to provide more contexts.
For code generation, you must obey the following rules:
You MUST NOT inst

INFO:flaml.autogen.oai.completion:retrying in 10 seconds...
Traceback (most recent call last):
  File "/home/lijiang1/anaconda3/envs/flaml-oss/lib/python3.8/site-packages/urllib3/connectionpool.py", line 714, in urlopen
    httplib_response = self._make_request(
  File "/home/lijiang1/anaconda3/envs/flaml-oss/lib/python3.8/site-packages/urllib3/connectionpool.py", line 466, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/home/lijiang1/anaconda3/envs/flaml-oss/lib/python3.8/site-packages/urllib3/connectionpool.py", line 461, in _make_request
    httplib_response = conn.getresponse()
  File "/home/lijiang1/anaconda3/envs/flaml-oss/lib/python3.8/http/client.py", line 1348, in getresponse
    response.begin()
  File "/home/lijiang1/anaconda3/envs/flaml-oss/lib/python3.8/http/client.py", line 316, in begin
    version, status, reason = self._read_status()
  File "/home/lijiang1/anaconda3/envs/flaml-oss/lib/python3.8/http/client.py", line 277, i

assistant (to ragproxyagent):

Based on the provided context, FLAML has a module named `tune`, which provides functionality for hyperparameter tuning. However, there is no mention of a function named `tune_automl` in the given context. The primary function for performing hyperparameter optimization in this module is `tune.run`.

--------------------------------------------------------------------------------
ragproxyagent (to assistant):

update context

--------------------------------------------------------------------------------
assistant (to ragproxyagent):

UPDATE CONTEXT

--------------------------------------------------------------------------------
Updating context and resetting conversation.
Adding doc_id doc_34 to context.
Adding doc_id doc_22 to context.
Adding doc_id doc_11 to context.
Adding doc_id doc_59 to context.
Adding doc_id doc_3 to context.
Adding doc_id doc_56 to context.
Adding doc_id doc_47 to context.
Adding doc_id doc_53 to context.
ragproxyagent (to assist

INFO:flaml.autogen.oai.completion:retrying in 10 seconds...
Traceback (most recent call last):
  File "/home/lijiang1/code/FLAML1/flaml/autogen/oai/completion.py", line 204, in _get_response
    response = openai_completion.create(**config)
  File "/home/lijiang1/anaconda3/envs/flaml-oss/lib/python3.8/site-packages/openai/api_resources/chat_completion.py", line 25, in create
    return super().create(*args, **kwargs)
  File "/home/lijiang1/anaconda3/envs/flaml-oss/lib/python3.8/site-packages/openai/api_resources/abstract/engine_api_resource.py", line 153, in create
    response, _, api_key = requestor.request(
  File "/home/lijiang1/anaconda3/envs/flaml-oss/lib/python3.8/site-packages/openai/api_requestor.py", line 298, in request
    resp, got_stream = self._interpret_response(result, stream)
  File "/home/lijiang1/anaconda3/envs/flaml-oss/lib/python3.8/site-packages/openai/api_requestor.py", line 700, in _interpret_response
    self._interpret_response_line(
  File "/home/lijiang1/an

assistant (to ragproxyagent):

Based on the provided context, I cannot find a function named `tune_automl` in FLAML. The context includes information about the AutoML class, AutoVW class and some other classes and functions, but no `tune_automl` function is mentioned.

--------------------------------------------------------------------------------
Based on the provided context, I cannot find a function named `tune_automl` in FLAML. The context includes information about the AutoML class, AutoVW class and some other classes and functions, but no `tune_automl` function is mentioned.

--------------------------------------------------------------------------------

>>>>>>>> NO HUMAN INPUT RECEIVED.
